In [10]:
# 导入所需的依赖包
import os
os.environ["TRANSFORMS_VERBOSITY"] = "error"
import sys
sys.path.append("..")
from config.load_key import load_key
from llama_index.core.node_parser import (
  SentenceSplitter,
  SemanticSplitterNodeParser,
  SentenceWindowNodeParser,
  MarkdownNodeParser,
  TokenTextSplitter
)

In [ ]:


def evaluate_splitter(splitter, documents, question, ground_truth, splitter_name):
  """评测不同文档切片方法的效果"""
  print(f"\n{'='*50}\n")
  print(f"🔍 正在使用 {splitter_name} 方法进行测试...")
  print(f"\n{'='*50}\n")

  # 构建索引
  print("📑 正在处理文档...")
  nodes = splitter.get_nodes_from_documents(documents)
  index = VectorStoreIndex(nodes, embed_model = Settings.embed_model)

  # 创建查询引擎
  query_engine = index.as_query_engine(
    similarity_top_k = 5,
    stream = True
  )

  # 执行查询
  print(f"\n❓ 测试问题: {question}")
  print("\n🤖 模型回答:")
  response = query_engine.query(question)
  response.print_response_stream()

  # 输出参考片段
  print(f"\n📚 {splitter_name} 召回的参考片段:")
  for i, node in enumerate(response.source_nodes, 1):
    print(f"\n文档片段 {i}:")
    print("-"*40)
    print(node)

  # 评估结果
  print(f"\n📊 {splitter_name} 评估结果:")
  print("-"*40)
  display(evaluate_result(question, response, ground_truth))